In [2]:
!pip install ipython-sql


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import sqlite3

In [4]:
df = pd.read_csv('order_details.csv')
df

,order_details_id,order_id,order_date,order_time,item_id
0,1,1,1/1/23,11:38:36 AM,109.0
1,2,2,1/1/23,11:57:40 AM,108.0
2,3,2,1/1/23,11:57:40 AM,124.0
3,4,2,1/1/23,11:57:40 AM,117.0
4,5,2,1/1/23,11:57:40 AM,129.0
...,...,...,...,...,...
12229,12230,5369,3/31/23,10:05:04 PM,109.0
12230,12231,5369,3/31/23,10:05:04 PM,129.0
12231,12232,5369,3/31/23,10:05:04 PM,120.0
12232,12233,5369,3/31/23,10:05:04 PM,122.0


In [42]:
# Create a connection to a sqlite database (It will create one if it doesn't exist)
conn = sqlite3.connect("example_2.db")

In [6]:
# Load the Dataframe into Sql table
df.to_sql('order_details', conn, if_exists='replace', index=False)

12234

In [7]:
conn.close()

In [8]:
%load_ext sql

In [43]:
%sql sqlite:///example_2.db

## 1. View the order details table.

In [11]:
%%sql

SELECT * FROM order_details LIMIT 10; 

 * sqlite:///example_1.db
Done.


order_details_id,order_id,order_date,order_time,item_id
1,1,1/1/23,11:38:36 AM,109.0
2,2,1/1/23,11:57:40 AM,108.0
3,2,1/1/23,11:57:40 AM,124.0
4,2,1/1/23,11:57:40 AM,117.0
5,2,1/1/23,11:57:40 AM,129.0
6,2,1/1/23,11:57:40 AM,106.0
7,3,1/1/23,12:12:28 PM,117.0
8,3,1/1/23,12:12:28 PM,119.0
9,4,1/1/23,12:16:31 PM,117.0
10,5,1/1/23,12:21:30 PM,117.0


In [12]:
%%sql

SELECT COUNT (*) FROM order_details;

 * sqlite:///example_1.db
Done.


COUNT (*)
12234


## 2. What is the date range of the table?

In [13]:
%%sql

SELECT MIN(order_date), MAX(order_date) FROM order_details;

 * sqlite:///example_1.db
Done.


MIN(order_date),MAX(order_date)
1/1/23,3/9/23


## 3. How many orders were made within this date range?

In [17]:
%%sql

SELECT COUNT(DISTINCT order_id) FROM order_details;

 * sqlite:///example_1.db
Done.


COUNT(DISTINCT order_id)
5370


In [19]:
%%sql

SELECT 
    COUNT(DISTINCT order_id) AS total_orders, 
    MIN(order_date) AS earliest_order, 
    MAX(order_date) AS latest_order
FROM order_details
WHERE order_date BETWEEN '1/1/23' AND '3/9/23';

 * sqlite:///example_1.db
Done.


total_orders,earliest_order,latest_order
5370,1/1/23,3/9/23


## 4. How many items were ordered within this date range?

In [20]:
%%sql

SELECT COUNT(*) FROM order_details;

 * sqlite:///example_1.db
Done.


COUNT(*)
12234


## 5. Which orders had the most number of items?

In [37]:
%%sql

SELECT order_id, COUNT(item_id) as num_items
FROM order_details
GROUP BY order_id 
ORDER BY num_items DESC LIMIT 20;

 * sqlite:///example_1.db
Done.


order_id,num_items
4305,14
3473,14
2675,14
1957,14
443,14
440,14
330,14
5200,13
5066,13
4836,13


## 6. How many orders had more than 12 items?

In [40]:
%%sql

SELECT order_id, COUNT(item_id) as num_items
FROM order_details
GROUP BY order_id
HAVING num_items > 12;

 * sqlite:///example_1.db
Done.


order_id,num_items
330,14
440,14
443,14
1274,13
1569,13
1685,13
1734,13
1957,14
2075,13
2126,13


In [41]:
%%sql

SELECT COUNT(*) FROM

(SELECT order_id, COUNT(item_id) as num_items
FROM order_details
GROUP BY order_id
HAVING num_items > 12) as num_orders;

 * sqlite:///example_1.db
Done.


COUNT(*)
20
